In [0]:
import os
from dotenv import load_dotenv
from datetime import datetime

from pyspark.sql import functions as F

In [0]:
load_dotenv()

DEFAULT_PATH = os.getenv("DEFAULT_PATH")

In [0]:
_current_datetime = dbutils.widgets.get('current_datetime')
_rate = dbutils.widgets.get('rate')

current_date = datetime.today().strftime('%Y-%m-%d')

raw_path = f"{DEFAULT_PATH}/Raw/{current_date}.json"
silver_path = f"{DEFAULT_PATH}/Silver/{current_date}.parquet"

In [0]:
df = spark.read.json(raw_path)

In [0]:

df = df.withColumn('base_currency', F.lit(_rate).cast('string'))\
       .withColumn('timestamp', F.lit(_current_datetime).cast('timestamp'))\
       .withColumnRenamed('currency', 'moeda')\
       .withColumnRenamed('value', 'taxa')\
       .withColumn('moeda', F.col('moeda').cast('string'))\
       .withColumn('taxa', F.col('taxa').cast('double'))\
       .filter(F.col('moeda') != _rate)\
       .filter((F.col('taxa') > 0) & (F.col('taxa').isNotNull()))



In [0]:
if os.path.exists(silver_path):
       df.write.mode('append').format('parquet').save(silver_path)
else:
       df.coalesce(1).write.mode('overwrite').format('parquet').save(silver_path)